In [2]:
import pandas as pd
import numpy as np

# Import Dataset

In [3]:
# Import RedcapExport.csv as redcap
redcap = pd.read_csv('Redcap Export.csv')
# Display the first 5 rows of the dataframe
redcap.head()

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,subjectid,subject_referral,covid_group,interventiongroup,subject_information_complete,mmrc_score,...,pdf_outcomemeasures,pdf_complete,diary_sessions_1,diary_sessions_2,diary_sessions_3,diary_sessions_4,diary_mip_1,diary_mip_2,diary_mip_3,training_diary_complete
0,1,screening_arm_1,NaN,NaN,1.0,Physical Therapy,1.0,NaN,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,visit_2_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,pcs01v2 measures.pdf,2.0,14.0,14.0,14.0,14.0,65.0,83.0,80.0,2.0
2,1,baseline_arm_1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,...,pcs01v1 measures.pdf,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,screening_arm_1,NaN,NaN,2.0,Physical Therapy,1.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,visit_2_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,pcs02v2 measures.pdf,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [4]:
# Step 1. Extract covid_group and interventiongroup from 'screening_arm_1' rows
# and map them to all rows with the same record_id

# Create a small DataFrame containing just screening_arm_1 rows
screening_data = redcap[redcap['redcap_event_name'] == 'screening_arm_1']

# Build a mapping dictionary for covid_group and interventiongroup
covid_group_map = screening_data.set_index('record_id')['covid_group'].to_dict()
interventiongroup_map = screening_data.set_index('record_id')['interventiongroup'].to_dict()

# Fill missing values in covid_group and interventiongroup columns across all rows
redcap['covid_group'] = redcap.apply(
    lambda row: covid_group_map.get(row['record_id'], row['covid_group']),
    axis=1
)
redcap['interventiongroup'] = redcap.apply(
    lambda row: interventiongroup_map.get(row['record_id'], row['interventiongroup']),
    axis=1
)

# Step 2. Remove 'screening_arm_1' rows
redcap = redcap[redcap['redcap_event_name'].isin(['baseline_arm_1', 'visit_2_arm_1'])]

# Step 3. Create the time_point column
time_point_map = {
    'baseline_arm_1': 1,
    'visit_2_arm_1': 2
}
redcap['time_point'] = redcap['redcap_event_name'].map(time_point_map)

# NOTE: Remember to update RedCAP (FIX THIS)
# Step 4. Manually overwrite interventiongroup using your dictionary
manual_interventiongroup = {
    1:1, 2:0, 3:1, 4:0, 5:0, 6:0, 7:1, 8:1, 9:0, 10:1,
    11:1, 13:1, 14:0, 15:1, 16:0, 17:0, 18:1, 20:0, 21:1
}

# Temporarily set Covid_group = 1 for record_id 7
redcap.loc[redcap['record_id'] == 7, 'covid_group'] = 1
# NOTE: Remember to update RedCAP (FIX THIS)


# Update interventiongroup
redcap['interventiongroup'] = redcap['record_id'].map(manual_interventiongroup).combine_first(redcap['interventiongroup'])

# Remove all columns where every value is NaN
redcap = redcap.dropna(axis=1, how='all')

# Drop unnecessary columns
redcap = redcap.drop(columns=['redcap_event_name', 'redcap_repeat_instance'])

# Remove all columns that end with '_complete'
redcap = redcap.loc[:, ~redcap.columns.str.endswith('_complete')]





In [11]:

# Export the cleaned DataFrame to a new CSV file
redcap.to_csv('cleaned_redcap_data.csv', index=False)

redcap.head(30)


,record_id,covid_group,interventiongroup,mmrc_score,pcfs_score,subject_dob,subject_gender,subject_ethnicity,health_ldl,health_hdl,...,pdf_compensation,pdf_outcomemeasures,diary_sessions_1,diary_sessions_2,diary_sessions_3,diary_sessions_4,diary_mip_1,diary_mip_2,diary_mip_3,time_point
1,1,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,pcs01v2 compensation.pdf,pcs01v2 measures.pdf,14.0,14.0,14.0,14.0,65.0,83.0,80.0,2
2,1,1.0,1.0,2.0,2.0,1992-04-16,Female,Latino,NaN,NaN,...,pcs01v1 compensation.pdf,pcs01v1 measures.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,pcs02v2 compensation.pdf,pcs02v2 measures.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
5,2,1.0,0.0,2.0,2.0,1940-03-13,Female,White,91.0,60.0,...,pcs02v1 compensation.pdf,pcs02v1 measures.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,3,1.0,1.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,...,pcs03v2 compensation.pdf,pcs03v2 measures.pdf,14.0,14.0,14.0,14.0,89.0,108.0,120.0,2
8,3,1.0,1.0,4.0,4.0,1976-11-24,Female,Hispanic,NaN,NaN,...,pcs03v1 compensation.pdf,pcs03v1 measures.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
10,4,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
11,4,1.0,0.0,1.0,2.0,1986-04-30,Male,White,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
13,5,1.0,0.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
14,5,1.0,0.0,3.0,3.0,1963-12-26,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Respiratory Strength

In [10]:
# Define measures and suffixes
measures = ['mip', 'fit', 'smip', 'pif', 'sindex', 'volume']
suffixes = ['max', 'min', 'mean']

# Loop through measures and suffixes
for measure in measures:
    for suffix in suffixes:
        pre_col = f"{measure}_pre_{suffix}"
        post_col = f"{measure}_post_{suffix}"

        if pre_col in redcap.columns and post_col in redcap.columns:
            # Fatigue: pre - post
            fatigue_col = f"{measure}_{suffix}_fatigue"
            redcap[fatigue_col] = redcap[pre_col] - redcap[post_col]

            # Fatigue Percent: (pre - post) / pre * 100
            fatigue_percent_col = f"{measure}_{suffix}_fatigue_percent"
            redcap[fatigue_percent_col] = ((redcap[pre_col] - redcap[post_col]) / redcap[pre_col]) * 100

        else:
            print(f"⚠️ Warning: Missing columns for {measure}_{suffix}")


In [11]:
redcap.head()

,record_id,covid_group,interventiongroup,mmrc_score,pcfs_score,subject_dob,subject_gender,subject_ethnicity,health_ldl,health_hdl,health_triglyceride,health_smoking,health_history,health_medications,covid_onset,covid_infections,covid_vaccinations,covid_supplemento2,covid_hospitalization,covid_icu,covid_mechanicalvent,covid_intubation,covid_apnea,covid_apnea_ahi,covid_apnea_cpap,covid_hallucinations,covid_taste,covid_smell,covid_hearing,covid_tinnitus,covid_vision,covid_throat,covid_choking,covid_sweating,covid_sneezing,covid_coughing,covid_voice,covid_fevers,covid_hair,covid_skin,covid_digestion,covid_chestpain,covid_palpitations,covid_restlessleg,covid_other,covid_primary,data_age,cpet_hr_restingsit,cpet_sbp_sitting,cpet_dbp_sitting,cpet_hr_restingstand,cpet_sbp_standing,cpet_dbp_standing,cpet_spo2_pre,data_kilograms,data_centimeters,data_bmi,data_gender,fmd_diameter_baseline,fmd_diameter_maximum,fmd_percent,fmd_shear_baseline,fmd_shear_maximum,fmd_shear_auc,fmd_shear_areatomaximum,fmd_velocity_baseline,fmd_velocity_maximum,fmd_csv,imt_mip_1_pre,imt_mip_1_post,imt_mip_2_pre,imt_mip_2_post,imt_mip_3_pre,imt_mip_3_post,imt_fit_1_pre,imt_fit_1_post,imt_fit_2_pre,imt_fit_2_post,imt_fit_3_pre,imt_fit_3_post,imt_smip_1_pre,imt_smip_1_post,imt_smip_2_pre,imt_smip_2_post,imt_smip_3_pre,imt_smip_3_post,imt_pif_1_pre,imt_pif_1_post,imt_pif_2_pre,imt_pif_2_post,imt_pif_3_pre,imt_pif_3_post,imt_sindex_1_pre,imt_sindex_1_post,imt_sindex_2_pre,imt_sindex_2_post,imt_sindex_3_pre,imt_sindex_3_post,imt_volume_1_pre,imt_volume_1_post,imt_volume_2_pre,imt_volume_2_post,imt_volume_3_pre,imt_volume_3_post,imt_mep_1,imt_mep_2,imt_mep_3,imt_meppower_1,imt_meppower_2,imt_meppower_3,imt_mepfit_1,imt_mepfit_2,imt_mepfit_3,cpet_hr_peak,cpet_sbp_peak,cpet_dbp_peak,cpet_borg,cpet_spo2_post,cpet_hr_cooldown1,cpet_hr_cooldown2,cpet_vo2peak_absolute,cpet_vo2peak_relative,cpet_vo2threshold,cpet_vevco2,cpet_rr_peak,cpet_vo2pulse,cpet_petco2,cpet_rer,cpet_chronindex,cpet_hrr1,cpet_hrr2,cpet_csv,hrv_sdnn,hrv_rmssd,hrv_heartrate,hrv_lf,hrv_hf,hrv_lfhf,hrv_csv,woods_concentration,woods_decisions,woods_confusion,woods_memory,woods_words,woods_takethingsin,woods_processingspeed,woods_thoughtsmixed,woods_muzzy,fss_motivation,fss_exercise,fss_easily,fss_functioning,fss_problems,fss_sustained,fss_duties,fss_disabling,fss_social,dsq_heavy_freq,dsq_nextday_freq,dsq_mentallytired_freq,dsq_minexercise_freq,dsq_drained_freq,dsq_heavy_severity,dsq_nextday_severity,dsq_mentallytired_severity,dsq_minexercise_severity,dsq_drained_severity,psqi_sleepstart,psqi_latency,psqi_sleepend,psqi_hours,psqi_latency30,psqi_wake,psqi_bathroom,psqi_breathe,psqi_snore,psqi_cold,psqi_hot,psqi_dreams,psqi_pain,psqi_other,psqi_medicine,psqi_sleepy,psqi_enthusiasm,psqi_quality,ndi_intensity,ndi_personalcare,ndi_lifting,ndi_work,ndi_headaches,ndi_concentration,ndi_sleeping,ndi_driving,ndi_reading,ndi_recreation,odi_intensity,odi_personalcare,odi_lifting,odi_walking,odi_sitting,odi_standing,odi_sleeping,odi_sex,odi_social,odi_traveling,vas_backpainworst,vas_backpainbest,vas_backpaincurrent,vas_neckpainworst,vas_neckpainbest,vas_neckpaincurrent,vas_myalgiaworst,vas_myalgiabest,vas_myalgiacurrent,vas_dizzyworst,vas_dizzybest,vas_dizzycurrent,vas_fatigueworst,vas_fatiguebest,vas_fatiguecurrent,gad_anxious,gad_worrying,phq_hopeless,phq_anhedonia,ptsd_nightmares,ptsd_intrusive,ptsd_startled,ptsd_detached,ptsd_guilty,sfpa_vigorous,sfpa_moderate,sfpa_lifting,sfpa_stairs2,sfpa_stairs1,sfpa_stooping,sfpa_walkingmile,sfpa_walkingblocks2,sfpa_walkingblocks1,sfpa_bathingdress,eq_vas,bdi_functional,bdi_task,bdi_effort,tdi_functional,tdi_task,tdi_effort,pdf_consent,pdf_compensation,pdf_outcomemeasures,diary_sessions_1,diary_sessions_2,diary_sessions_3,diary_sessions_4,diary_mip_1,diary_mip_2,diary_mip_3,time_point,mip_pre_max,mip_pre_min,mip_pre_mean,mip_post_max,mip_post_min,mip_post_mean,fit_pre_max,fit_pre_min,fit_pre_mean,fit_post_max,fit_post_min,fit_post_mean,smip_pre_max,smip_pre_min,smip_pre_mean,smip_p

# Subjective Questionnaires

## Woods Mental Fatigue Inventory

In [13]:
# List of columns to sum
woods_columns = [
    'woods_concentration', 'woods_decisions', 'woods_confusion',
    'woods_memory', 'woods_words', 'woods_takethingsin',
    'woods_processingspeed', 'woods_thoughtsmixed', 'woods_muzzy'
]

# Create the new column 'woods_sum'
redcap['woods_sum'] = redcap[woods_columns].sum(axis=1)

# Display mean, min, max, and std of the 'woods_sum' column
print("Woods Sum Statistics:")
print(f"Mean: {redcap['woods_sum'].mean()}")
print(f"Min: {redcap['woods_sum'].min()}")
print(f"Max: {redcap['woods_sum'].max()}")
print(f"Std: {redcap['woods_sum'].std()}")


Woods Sum Statistics:
Mean: 16.483870967741936
Min: 1.0
Max: 31.0
Std: 8.958686539673606


## Fatigue Severity Scale (FSS)

In [14]:
# List of columns to sum
fss_columns = ['fss_motivation', 'fss_exercise', 'fss_easily', 'fss_functioning',
       'fss_problems', 'fss_sustained', 'fss_duties', 'fss_disabling',
       'fss_social']

# Create the new column 'fss_sum'
redcap['fss_sum'] = redcap[fss_columns].sum(axis=1)

# Create a new column fss_dichotomous if fss_sum is greater than 36
redcap['fss_dichotomous'] = redcap['fss_sum'].apply(lambda x: 1 if x > 36 else 0)

# Print the mean, min, max, and std of the 'fss_sum' column
print("FSS Sum Statistics:")
print(f"Mean: {redcap['fss_sum'].mean()}")
print(f"Min: {redcap['fss_sum'].min()}")
print(f"Max: {redcap['fss_sum'].max()}")
print(f"Std: {redcap['fss_sum'].std()}")

FSS Sum Statistics:
Mean: 38.67741935483871
Min: 10.0
Max: 62.0
Std: 16.682100380895672


## PEM (DSQ)

In [15]:
# https://pmc.ncbi.nlm.nih.gov/articles/PMC6165517/
# A frequency and severity score of 2, 2 on any items 1–5 is indicative of PEM.

# Lists of columns
freq_cols = [
    'dsq_heavy_freq', 'dsq_nextday_freq', 'dsq_mentallytired_freq',
    'dsq_minexercise_freq', 'dsq_drained_freq'
]

severity_cols = [
    'dsq_heavy_severity', 'dsq_nextday_severity', 'dsq_mentallytired_severity',
    'dsq_minexercise_severity', 'dsq_drained_severity'
]

# Sum of frequency
redcap['dsq_freq_sum'] = redcap[freq_cols].sum(axis=1)

# Sum of severity
redcap['dsq_severity_sum'] = redcap[severity_cols].sum(axis=1)

# Sum of both frequency and severity
redcap['dsq_sum'] = redcap['dsq_freq_sum'] + redcap['dsq_severity_sum']

# Dichotomous indicator (1 if any item has freq >=2 AND severity >=2)
def compute_dsq_dichotomous(row):
    for freq_col, sev_col in zip(freq_cols, severity_cols):
        if (row[freq_col] >= 2) and (row[sev_col] >= 2):
            return 1
    return 0

redcap['dsq_dichotomous'] = redcap.apply(compute_dsq_dichotomous, axis=1)

# Print the frequency counts for dsq_dichotomous
print("DSQ Dichotomous Frequency Counts:")
print(redcap['dsq_dichotomous'].value_counts())

# Print the mean, min, max, and std of the 'dsq_sum' column
print("DSQ Sum Statistics:")
print(f"Mean: {redcap['dsq_sum'].mean()}")
print(f"Min: {redcap['dsq_sum'].min()}")
print(f"Max: {redcap['dsq_sum'].max()}")



DSQ Dichotomous Frequency Counts:
dsq_dichotomous
1    19
0    12
Name: count, dtype: int64
DSQ Sum Statistics:
Mean: 15.290322580645162
Min: 0.0
Max: 38.0


## Pittsburgh Sleep Index (PSQI)

In [16]:
# Keep the raw Q2 latency in a separate column (assuming it's the original minutes)
# If 'psqi_latency' currently holds raw minutes, just rename it to 'psqi_latency_raw'
redcap['psqi_latency_raw'] = redcap['psqi_latency']

# Recode Q2 (raw latency in minutes) into 0-3 scale
def recode_q2(x):
    if pd.isnull(x):
        return np.nan
    elif x < 15:
        return 0
    elif 15 <= x <= 30:
        return 1
    elif 31 <= x <= 60:
        return 2
    else:  # > 60 minutes
        return 3

redcap['psqi_latency_q2'] = redcap['psqi_latency_raw'].apply(recode_q2)

# Q5a is already coded as 0-3, keep it as is in 'psqi_latency30'
redcap['psqi_latency_q5a'] = redcap['psqi_latency30']

# Sum the two recoded latency subscores
redcap['psqi_latency_sum'] = redcap[['psqi_latency_q2', 'psqi_latency_q5a']].sum(axis=1)

# Map the summed value into the final component score
def map_latency_component(x):
    if pd.isnull(x):
        return np.nan
    elif x == 0:
        return 0
    elif 1 <= x <= 2:
        return 1
    elif 3 <= x <= 4:
        return 2
    else:  # 5-6
        return 3

redcap['psqi_latency'] = redcap['psqi_latency_sum'].apply(map_latency_component)

# 3. Sleep Duration
def score_sleep_duration(hours):
    if hours >= 7:
        return 0
    elif 6 <= hours < 7:
        return 1
    elif 5 <= hours < 6:
        return 2
    else:
        return 3

redcap['psqi_duration'] = redcap['psqi_hours'].apply(score_sleep_duration)

# 4. Sleep Efficiency
def hhmm_to_decimal(time_val):
    """
    Converts HHMM integer to decimal hours.
    E.g. 2230 -> 22 + 30/60 = 22.5
    """
    if pd.isnull(time_val):
        return np.nan
    hours = time_val // 100
    minutes = time_val % 100
    return hours + minutes / 60.0

# --- Convert psqi_sleepstart and psqi_sleepend to decimal hours ---
redcap['sleepstart_decimal'] = redcap['psqi_sleepstart'].apply(hhmm_to_decimal)
redcap['sleepend_decimal'] = redcap['psqi_sleepend'].apply(hhmm_to_decimal)

# --- Calculate Time in Bed (handling overnight shifts) ---
def calculate_time_in_bed(start, end):
    if pd.isnull(start) or pd.isnull(end):
        return np.nan
    time_in_bed = end - start
    if time_in_bed <= 0:
        time_in_bed += 24
    return time_in_bed

redcap['time_in_bed'] = redcap.apply(
    lambda row: calculate_time_in_bed(row['sleepstart_decimal'], row['sleepend_decimal']), axis=1
)

# --- Calculate Sleep Efficiency ---
redcap['sleep_efficiency'] = (redcap['psqi_hours'] / redcap['time_in_bed']) * 100

# --- Score PSQI Component 4: Habitual Sleep Efficiency ---
def score_sleep_efficiency(efficiency):
    if pd.isnull(efficiency):
        return np.nan
    elif efficiency >= 85:
        return 0
    elif 75 <= efficiency < 85:
        return 1
    elif 65 <= efficiency < 75:
        return 2
    else:
        return 3

redcap['psqi_efficiency'] = redcap['sleep_efficiency'].apply(score_sleep_efficiency)

# 5. Sleep Disturbances
disturbance_items = [
    'psqi_wake', 'psqi_bathroom', 'psqi_breathe', 'psqi_snore',
    'psqi_cold', 'psqi_hot', 'psqi_dreams', 'psqi_pain', 'psqi_other'
]

# TEMPORARY FIX: NEED TO REMOVE LATER AND FIX DATA ENTRY
redcap['psqi_other'] = pd.to_numeric(redcap['psqi_other'], errors='coerce').fillna(0)

redcap['psqi_disturbances_raw'] = redcap[disturbance_items].sum(axis=1)

redcap['psqi_disturbances'] = pd.cut(
    redcap['psqi_disturbances_raw'],
    bins=[-1, 0, 9, 18, 27],  # Adjusted bins
    labels=[0, 1, 2, 3]
).astype(int)


# 6. Use of Sleep Medications
redcap['psqi_medication'] = redcap['psqi_medicine']

# 7. Daytime Dysfunction
redcap['psqi_dysfunction_raw'] = redcap[['psqi_sleepy', 'psqi_enthusiasm']].sum(axis=1)
redcap['psqi_dysfunction'] = pd.cut(
    redcap['psqi_dysfunction_raw'],
    bins=[-1, 0, 2, 4, 6],
    labels=[0, 1, 2, 3]
).astype(int)

# 8. Calculate PSQI Sum
component_cols = [
    'psqi_quality', 'psqi_latency', 'psqi_duration', 
    'psqi_efficiency', 'psqi_disturbances', 
    'psqi_medication', 'psqi_dysfunction'
]
redcap['psqi_sum'] = redcap[component_cols].sum(axis=1)

# 9. Dichotomous classification: 0 = Good sleep, 1 = Poor sleep
redcap['psqi_dichotomous'] = (redcap['psqi_sum'] > 5).astype(int)


# List of all component columns plus the sum
components = [
    'psqi_quality', 'psqi_latency', 'psqi_duration', 'psqi_efficiency',
    'psqi_disturbances', 'psqi_medication', 'psqi_dysfunction', 'psqi_sum'
]

# Summary stats: mean, min, max
summary_stats = redcap[components].agg(['mean', 'min', 'max'])
print("Summary statistics for PSQI components and sum:")
print(summary_stats)

# Frequency counts for the dichotomous variable
dichotomous_counts = redcap['psqi_dichotomous'].value_counts(dropna=False)
print("\nFrequency counts for psqi_dichotomous:")
print(dichotomous_counts)


Summary statistics for PSQI components and sum:
      psqi_quality  psqi_latency  psqi_duration  psqi_efficiency  \
mean      1.387097      1.419355       0.935484          0.83871   
min       0.000000      0.000000       0.000000          0.00000   
max       3.000000      3.000000       3.000000          3.00000   

      psqi_disturbances  psqi_medication  psqi_dysfunction  psqi_sum  
mean           1.483871         0.935484           1.16129   8.16129  
min            0.000000         0.000000           0.00000   1.00000  
max            3.000000         3.000000           3.00000  18.00000  

Frequency counts for psqi_dichotomous:
psqi_dichotomous
1    21
0    10
Name: count, dtype: int64


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\1881106084.py:112: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['psqi_disturbances'] = pd.cut(
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\1881106084.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['psqi_medication'] = redcap['psqi_medicine']
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\1881106084.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

## BDI-TDI

In [19]:
# Compute BDI Score
redcap['bdi_sum'] = redcap[['bdi_functional', 'bdi_task', 'bdi_effort']].sum(axis=1)

# Compute TDI Score
redcap['tdi_sum'] = redcap[['tdi_functional', 'tdi_task', 'tdi_effort']].sum(axis=1)


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\4060981269.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['bdi_sum'] = redcap[['bdi_functional', 'bdi_task', 'bdi_effort']].sum(axis=1)
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\4060981269.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['tdi_sum'] = redcap[['tdi_functional', 'tdi_task', 'tdi_effort']].sum(axis=1)


## ODI / NDI

In [21]:
# Sum ODI columns
odi_cols = [
    'odi_intensity', 'odi_personalcare', 'odi_lifting', 'odi_walking',
    'odi_sitting', 'odi_standing', 'odi_sleeping', 'odi_sex',
    'odi_social', 'odi_traveling'
]
redcap['odi_sum'] = redcap[odi_cols].sum(axis=1, skipna=True)

# Sum NDI columns
ndi_cols = [
    'ndi_intensity', 'ndi_personalcare', 'ndi_lifting', 'ndi_work',
    'ndi_headaches', 'ndi_concentration', 'ndi_sleeping',
    'ndi_driving', 'ndi_reading', 'ndi_recreation'
]
redcap['ndi_sum'] = redcap[ndi_cols].sum(axis=1, skipna=True)


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\1534113519.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['odi_sum'] = redcap[odi_cols].sum(axis=1, skipna=True)
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\1534113519.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['ndi_sum'] = redcap[ndi_cols].sum(axis=1, skipna=True)


## Psychology Questionnaire

In [23]:
# Define the columns for each measure
anxiety_cols = ['gad_anxious', 'gad_worrying']
depression_cols = ['phq_hopeless', 'phq_anhedonia']
ptsd_cols = ['ptsd_nightmares', 'ptsd_intrusive', 'ptsd_startled', 'ptsd_detached', 'ptsd_guilty']

# Calculate sum scores
redcap['anxiety_sum'] = redcap[anxiety_cols].sum(axis=1, skipna=True)
redcap['depression_sum'] = redcap[depression_cols].sum(axis=1, skipna=True)
redcap['ptsd_sum'] = redcap[ptsd_cols].sum(axis=1, skipna=True)

# Create dichotomous variables
# PTSD-5: 3 most sensitive, 5 most specific, 4 most efficient https://pmc.ncbi.nlm.nih.gov/articles/PMC5023594/ 
# PHq-2: >=2 --> https://pubmed.ncbi.nlm.nih.gov/33026888/, https://jamanetwork.com/journals/jama/fullarticle/2766865
# GAD-2: >=3 --> https://www.sciencedirect.com/science/article/abs/pii/S0003999318303903, https://www.sciencedirect.com/science/article/abs/pii/S0163834315002406, https://pmc.ncbi.nlm.nih.gov/articles/PMC6163062/, https://pmc.ncbi.nlm.nih.gov/articles/PMC7306644/
redcap['anxiety_dichotomous'] = (redcap['anxiety_sum'] >= 3).astype(int)
redcap['depression_dichotomous'] = (redcap['depression_sum'] >= 2).astype(int)
redcap['ptsd_dichotomous'] = (redcap['ptsd_sum'] >= 3).astype(int)


C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\3429339521.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['anxiety_sum'] = redcap[anxiety_cols].sum(axis=1, skipna=True)
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\3429339521.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  redcap['depression_sum'] = redcap[depression_cols].sum(axis=1, skipna=True)
C:\Users\ZCooper\AppData\Local\Temp\ipykernel_21164\3429339521.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin

## SF-PA

In [26]:
# Define the columns for SFPA
sfpa_cols = [
    'sfpa_vigorous', 'sfpa_moderate', 'sfpa_lifting',
    'sfpa_stairs2', 'sfpa_stairs1', 'sfpa_stooping',
    'sfpa_walkingmile', 'sfpa_walkingblocks2',
    'sfpa_walkingblocks1', 'sfpa_bathingdress'
]

# Calculate the sum score
redcap['sfpa_sum'] = redcap[sfpa_cols].sum(axis=1, skipna=True)

# Export

In [25]:
redcap.head(30)

,record_id,covid_group,interventiongroup,mmrc_score,pcfs_score,subject_dob,subject_gender,subject_ethnicity,health_ldl,health_hdl,...,sindex_pre_mean,sindex_post_max,sindex_post_min,sindex_post_mean,volume_pre_max,volume_pre_min,volume_pre_mean,volume_post_max,volume_post_min,volume_post_mean
1,1,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,58.666667,63.0,59.0,61.333333,2.1,1.2,1.666667,2.5,0.7,1.300000
2,1,1.0,1.0,2.0,2.0,1992-04-16,Female,Latino,NaN,NaN,...,49.000000,52.0,44.0,48.333333,2.7,2.5,2.600000,3.0,2.9,2.933333
4,2,1.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,42.333333,51.0,47.0,49.333333,1.9,1.1,1.566667,1.7,1.5,1.633333
5,2,1.0,0.0,2.0,2.0,1940-03-13,Female,White,91.0,60.0,...,53.666667,49.0,41.0,45.000000,1.8,1.3,1.600000,2.3,1.9,2.133333
7,3,1.0,1.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,...,67.000000,82.0,77.0,79.666667,2.0,1.6,1.733333,2.6,2.2,2.433333
8,3,1.0,1.0,4.0,4.0,1976-11-24,Female,Hispanic,NaN,NaN,...,58.333333,52.0,39.0,46.000000,2.7,1.3,2.033333,2.7,2.5,2.633333
10,4,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,99.666667,114.0,96.0,104.000000,3.4,3.0,3.166667,3.1,2.3,2.700000
11,4,1.0,0.0,1.0,2.0,1986-04-30,Male,White,NaN,NaN,...,67.000000,95.0,89.0,91.333333,3.2,2.5,2.933333,3.3,2.8,3.100000
13,5,1.0,0.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,...,48.666667,49.0,34.0,42.000000,3.3,2.8,3.033333,3.0,2.7,2.866667
14,5,1.0,0.0,3.0,3.0,1963-12-26,NaN,NaN,NaN,NaN,...,42.666667,53.0,44.0,47.333333,3.7,0.3,2.333333,3.8,0.5,2.566667


In [27]:
# Create DataFrame for time_point = 1
df_CS = redcap[redcap['time_point'] == 1].copy()

# Create DataFrame for Covid_group = 1
df_RCT = redcap[redcap['covid_group'] == 1].copy()

# Export to CSV files
df_CS.to_csv('df_CS.csv', index=False)
df_RCT.to_csv('df_RCT.csv', index=False)
